# 3: Streams 

In [1]:
# Essential imports
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt


In [2]:
# Non-essential imports
# Wider notebook:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Fancy figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['lines.markersize'] = 10

# 3.1: Streams in a spherical potential  

Insert a particle at the centre of the system, i.e. located at (x,v) = (0,0) [kpc, km/s]. Next put the system on the orbit specified below and integrate the system (i.e. the 10$^4$ + 1 particles) for 5 Gyr in an NFW potential with $M = 0.8\cdot10^{12}~{\rm M}_\odot$ and $r_s = 16$ kpc


Plot the spatial evolution of the system in two planes (e.g. (x,y) and (x,z)) and at various times (e.g. 0,1,2,3,4, and 5 Gyr)  
- Comment on the behaviour of the particles as time goes by. What do you see happens? Can you explain qualitatively the reason for the behaviour observed?
- What are the differences in the behaviour between the orbits? 
- How would you explain the differing behaviours?

In a sphere of radius 1 kpc centred on the central particle, inspect and discuss: 
- the number density of particles inside this volume as a function of time
- the total velocity dispersion of the same particles as a function of time $(\sigma_v = \sqrt{\sigma_{vx}^2+\sigma_{vy}^2+\sigma_{vz}^2})$    

Hint : overplot the central orbit on the spatial evolution and use it to explain what you see.  


In [3]:
# Orbit NFW
pos = [8.0,0,8.0] # kpc
vel = [0,150,0] # km/s 

# 3.2: Streams in a Galactic potential

Repeat the analysis Section 3. but integrate the system that you generated for 5 Gyr in the Galactic potential implemented in gala `gala.potential.MilkyWayPotential`.  
Put the system on the orbits specified below (`Orbit 1` is the same as `Orbit NFW` from above).

Plot the spatial evolution of the system in two planes (e.g. (x,y) and (x,z)) and at various times (e.g. 0,1,2,3,4, and 5 Gyr)  
- Comment on the behaviour of the particles as time goes by. What do you see happens? Can you explain qualitatively the reason for the behaviour observed?
- What are the differences in the behaviour between the orbits? 
- How would you explain the differing behaviours?  

In a sphere of radius 1 kpc centred on the central particle, inspect and discuss: 
- the number density of particles inside this volume as a function of time
- the total velocity dispersion of the same particles as a function of time $(\sigma_v = \sqrt{\sigma_{vx}^2+\sigma_{vy}^2+\sigma_{vz}^2})$

At some intermediate time, once a stream has clearly developed, identify (visually) a particle in the leading arm of the stream and one in the trailing arm. Compute their total orbital energy.  
How do they differ? How do they compare to the energy of the central particle?  
Why do you think their energy is related to their location in a leading or trailing arm?

Hint 1: gala can compute the orbital energy using `orbit.energy()`.  
Hint 2: overplot the central orbit on the spatial evolution and use it to explain what you see.  
Hint 3: The chaotic orbit might take a long time with the `DOP853` integrator (see below for details).

For `Orbit 1`, what are the difference that you find w.r.t. the NFW potential?  
What drives these differences?


In [4]:
# Orbit 1 - Regular  [the units are kpc and km/s, respectively for positions and velocities]
pos = [8.0,0,8.0]
vel = [0,150,0]

# Orbit 2 - Chaotic
pos = [7.7,0.,-2.3]
vel = [-66,2,98]

# Orbit 3 - Regular (radial)
pos = [8.0,0,8.0]
vel = [0,0,0]